Primeiro, vamos preparar o ambiente e criar uma coleção otimizada para escala (como discutido no Day 4):

In [ ]:
import numpy as np
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, VectorParams, Batch, BinaryQuantization, BinaryQuantizationConfig

client = QdrantClient(":memory:") # Usando memória para o exemplo, mas seria a URL do seu servidor

# 1. Configuração Otimizada (Essencial para o Day 4)
client.recreate_collection(
    collection_name="escala_demo",
    vectors_config=VectorParams(size=128, distance=Distance.COSINE, on_disk=True), # Vetores no disco
    quantization_config=BinaryQuantization(
        binary=BinaryQuantizationConfig(always_ram=True) # Busca rápida na RAM
    )
)

Abordagem 1: upsert com Batches (Até 100k pontos)

Nesta abordagem, nós mesmos dividimos os dados em "pacotes" e enviamos. É o método mais flexível.

In [ ]:
def ingest_with_upsert(client, collection_name, total_points=1000):
    batch_size = 100
    for i in range(0, total_points, batch_size):
        # Criando um lote (batch) de dados
        ids = list(range(i, i + batch_size))
        vectors = np.random.rand(batch_size, 128).tolist()
        payloads = [{"original_id": j, "tipo": "teste"} for j in ids]

        client.upsert(
            collection_name=collection_name,
            points=Batch(
                ids=ids,
                vectors=vectors,
                payloads=payloads
            )
        )
    print(f"Ingestão de {total_points} concluída com upsert.")

ingest_with_upsert(client, "escala_demo")

Abordagem 2: upload_points (De 100k a 1M pontos)

Este método é um "atalho" do cliente Python que gerencia os lotes internamente para você. É mais limpo, mas os dados precisam estar na memória do seu script.

In [ ]:
from qdrant_client.models import PointStruct

def ingest_with_upload_points(client, collection_name, total_points=1000):
    # Geramos todos os pontos primeiro (consome RAM no seu PC)
    points = [
        PointStruct(
            id=i,
            vector=np.random.rand(128).tolist(),
            payload={"meta": "dados_medios"}
        ) for i in range(1000, 1000 + total_points)
    ]

    client.upload_points(
        collection_name=collection_name,
        points=points,
        batch_size=100,
        parallel=2 # Começamos a usar paralelismo aqui
    )
    print(f"Ingestão de {total_points} concluída com upload_points.")

ingest_with_upload_points(client, "escala_demo")

Abordagem 3: upload_collection (O padrão ouro para > 1M pontos)

Esta é a forma mais eficiente. Usamos um Gerador (Generator). Os dados não são carregados todos de uma vez; eles "fluem" do disco/banco de dados direto para o Qdrant.

In [ ]:
def data_generator(total_points):
    """Simula a leitura de um arquivo gigante linha por linha"""
    for i in range(5000, 5000 + total_points):
        yield PointStruct(
            id=i,
            vector=np.random.rand(128).tolist(),
            payload={"status": "streaming"}
        )

def ingest_large_scale(client, collection_name):
    # O método upload_collection aceita um iterador/gerador
    client.upload_collection(
        collection_name=collection_name,
        points=data_generator(10000), # Pode ser milhões aqui
        batch_size=256,
        parallel=4, # Usa 4 núcleos do processador para enviar em paralelo
    )
    print("Ingestão de larga escala concluída com streaming e paralelismo.")

ingest_large_scale(client, "escala_demo")

## Resumo

Resumo das Diferenças no Código:
Memória: No upload_points, você passa uma list[] (pesado). 
No upload_collection, você passa um generator (leve).

Velocidade: O parâmetro parallel=N no upload_collection é o que realmente faz a diferença em larga escala, pois abre múltiplas conexões simultâneas com o servidor Qdrant.

Configuração do Servidor: Note que no início usamos on_disk=True. Sem isso, se você tentar subir 100 milhões de pontos, o servidor Qdrant vai dar erro de Out of Memory (OOM), independentemente de quão bom seja seu código Python.